In [1]:
#Usunąć # przy pobieraniu

In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from textblob import TextBlob
from textblob import Word
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
#Delete unnecesary column and chech length of messeage
normal = pd.read_csv('spam.csv', encoding = 'latin-1')
normal.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
normal.columns = ['label', 'content']

In [4]:
#stats

In [5]:
#Length of message
normal['char_count'] = normal['content'].apply(len)
#Counting words
normal['word_count'] = normal['content'].apply(lambda x: len(str(x).split(" ")))

In [6]:
#Length of average word in message
def avg_word_len(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

normal['avg_word_len'] = normal['content'].apply(lambda x: avg_word_len(x))

In [7]:
#Counting stopwords
#nltk.download('stopwords')
stop = stopwords.words('english')

normal['stopwords_count'] = normal['content'].apply(lambda x : len([x for x in x.split() if x in stop]))

In [8]:
#Counting specials and  numeric chars
normal['specials_count'] = normal['content'].apply(lambda x: len(re.sub('[\w]+', '', x)))
normal['numerics_count'] = normal['content'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

#Counting words with only upper letters
normal['uppers_count'] = normal['content'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

normal.head(10)

,label,content,char_count,word_count,avg_word_len,stopwords_count,specials_count,numerics_count,uppers_count
0,ham,"Go until jurong point, crazy.. Available only ...",111,20,4.600000,4,28,0,0
1,ham,Ok lar... Joking wif u oni...,29,6,4.000000,0,11,0,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,28,4.571429,5,33,2,2
3,ham,U dun say so early hor... U c already then say...,49,11,3.545455,2,16,0,2
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,13,3.769231,5,14,0,1
5,spam,FreeMsg Hey there darling it's been 3 week's n...,148,32,3.656250,13,40,1,0
6,ham,Even my brother is not like to speak with me. ...,77,16,3.875000,6,17,0,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,160,26,5.192308,9,31,0,0
8,spam,WINNER!! As a valued network customer you have...,158,26,5.115385,5,32,1,2
9,spam,Had your mobile 11 months or more? U R entitle...,154,29,4.344828,8,30,2,3


In [9]:
#tu zrobić wykresy wstepne

In [10]:
#Text processing

In [11]:
#save date before processing to comparize
normal['backup'] = normal['content']

#toLower
normal['content'] = normal['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#remove specials
normal['content'] = normal['content'].str.replace('[^\w\s]','')
#remove stopwords
normal['content'] = normal['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

<ipython-input-11-5b82003daf36>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  normal['content'] = normal['content'].str.replace('[^\w\s]','')


In [12]:
#remove often used words
freq = pd.Series(' '.join(normal['content']).split()).value_counts()[:10]
freq

u       1119
call     576
2        478
im       462
get      386
ur       384
4        287
dont     279
go       278
ok       277
dtype: int64

In [13]:
freq = list(freq.index)
normal['content'] = normal['content'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
normal['content'].head()

0    jurong point crazy available bugis n great wor...
1                                   lar joking wif oni
2    free entry wkly comp win fa cup final tkts 21s...
3                      dun say early hor c already say
4               nah think goes usf lives around though
Name: content, dtype: object

In [14]:
#remove rarely used words
freq = pd.Series(' '.join(normal['content']).split()).value_counts()[-10:]
freq

funs               1
internetservice    1
janinexx           1
officethenampet    1
hvae               1
virtual            1
09050001295        1
corrct             1
arises             1
categories         1
dtype: int64

In [15]:
freq = list(freq.index)
normal['content'] = normal['content'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
normal['content'].head()

0    jurong point crazy available bugis n great wor...
1                                   lar joking wif oni
2    free entry wkly comp win fa cup final tkts 21s...
3                      dun say early hor c already say
4               nah think goes usf lives around though
Name: content, dtype: object

In [16]:
#repair messages
#normal['content'][:5].apply(lambda x: str(TextBlob(x).correct()))

In [17]:
#Stemming
#st = PorterStemmer()
#normal['content'] = normal['content'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [18]:
#Tokenization
#nltk.download('punkt')
TextBlob(normal['content'][1]).words

WordList(['lar', 'joking', 'wif', 'oni'])

In [19]:
#Lemmatisation
#nltk.download('wordnet')
normal['content'] = normal['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [20]:
normal[['content', 'backup']].head(50)

,content,backup
0,jurong point crazy available bugis n great wor...,"Go until jurong point, crazy.. Available only ..."
1,lar joking wif oni,Ok lar... Joking wif u oni...
2,free entry wkly comp win fa cup final tkts 21s...,Free entry in 2 a wkly comp to win FA Cup fina...
3,dun say early hor c already say,U dun say so early hor... U c already then say...
4,nah think go usf life around though,"Nah I don't think he goes to usf, he lives aro..."
5,freemsg hey darling 3 week word back id like f...,FreeMsg Hey there darling it's been 3 week's n...
6,even brother like speak treat like aid patent,Even my brother is not like to speak with me. ...
7,per request melle melle oru minnaminunginte nu...,As per your request 'Melle Melle (Oru Minnamin...
8,winner valued network customer selected receiv...,WINNER!! As a valued network customer you have...
9,mobile 11 month r entitled update latest colou...,Had your mobile 11 months or more? U R entitle...


In [21]:
#Vectorisation
toSave = normal

toSave.drop(['backup', 'char_count', 'word_count', 'avg_word_len', 'stopwords_count', 'specials_count', 'numerics_count', 'uppers_count'], axis=1, inplace=True)

toSave.head()

,label,content
0,ham,jurong point crazy available bugis n great wor...
1,ham,lar joking wif oni
2,spam,free entry wkly comp win fa cup final tkts 21s...
3,ham,dun say early hor c already say
4,ham,nah think go usf life around though


In [22]:
toSave.to_csv('spam_clean.csv', index = False)

glove_input = ('spam_clean.csv')
word2vec_output = ('spam_clean.csv.word2vec')
glove2word2vec(glove_input, word2vec_output)

<ipython-input-22-c981204e43a3>:5: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input, word2vec_output)


ValueError: could not convert string to float: 'point'